# DL CNN project

In [14]:
import os
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image

from collections import Counter

from sklearn.metrics import classification_report, confusion_matrix

from google.colab import files
from google.colab.patches import cv2_imshow

## Set up the Kaggle API

In [3]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

KeyboardInterrupt: 

## Download the dataset

In [ ]:
!kaggle datasets download "lakshaymiddha/crack-segmentation-dataset"
!unzip crack-segmentation-dataset.zip
!rm crack-segmentation-dataset.zip

## Dataset

In [15]:
# Paths
TRAIN_IMG_DIR = 'crack_segmentation_dataset/train/images'
TRAIN_MASK_DIR = 'crack_segmentation_dataset/train/masks'
TEST_IMG_DIR = 'crack_segmentation_dataset/test/images'
TEST_MASK_DIR = 'crack_segmentation_dataset/test/masks'

# Dataset Instance
class CrackSegmentationDatasetInstance:
    def __init__(self, filename, test=False):
        self.img_path = os.path.join(
            TEST_IMG_DIR if test else TRAIN_IMG_DIR, filename
        )
        self.is_crack = not filename.startswith('noncrack')
        self.mask_path = os.path.join(
            TEST_MASK_DIR if test else TRAIN_MASK_DIR, filename
        )

    def get_img(self):
        img = cv2.imread(self.img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img / 255.0  # Normalize to [0, 1]

    def get_label(self):
        return int(self.is_crack)  # Binary label: 0 or 1

    def get_mask(self):
        mask = cv2.imread(self.mask_path, cv2.IMREAD_GRAYSCALE)
        # The line below is questionable. Is this necessary?
        mask = cv2.resize(mask, (128, 128))  # Resize masks for segmentation task
        return mask / 255.0  # Normalize mask to [0, 1]

# Main Dataset
class CrackSegmentationDataset:
    def __init__(self):
        self.train_instances = [
            CrackSegmentationDatasetInstance(filename)
            for filename in os.listdir(TRAIN_IMG_DIR)
        ]
        self.test_instances = [
            CrackSegmentationDatasetInstance(filename, test=True)
            for filename in os.listdir(TEST_IMG_DIR)
        ]

    def _generator(self, instances, task='classification'):
        for instance in instances:
            img = instance.get_img()
            if task == 'classification':
                yield img, instance.get_label()
            elif task == 'segmentation':
                yield img, instance.get_mask()

    def get_dataset(self, split='train', batch_size=32, task='classification'):
        instances = (
            self.train_instances if split == 'train' else self.test_instances
        )
        dataset = tf.data.Dataset.from_generator(
            lambda: self._generator(instances, task),
            output_signature=(
                tf.TensorSpec(shape=(448, 448, 3), dtype=tf.float32),
                tf.TensorSpec(shape=(), dtype=tf.int32 if task == 'classification' else tf.float32),
            )
        )
        if task == 'segmentation':
            dataset = dataset.map(lambda x, y: (x, tf.expand_dims(y, axis=-1)))
        dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
        return dataset

In [16]:
dataset = CrackSegmentationDataset()

## Dataset analysis

In [13]:
counter = Counter(instance.get_label() for instance in dataset.train_instances)

print(f"Number of non-crack images: {counter[0]}")
print(f"Number of crack images: {counter[1]}")

Number of non-crack images: 1199
Number of crack images: 8404


## Basic classification

In [7]:
# Binary classification dataset
train_ds = dataset.get_dataset(split='train', batch_size=32, task='classification')
test_ds = dataset.get_dataset(split='test', batch_size=32, task='classification')

# Build simple binary classification model
classification_model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(448, 448, 3)),  # Define fixed input shape
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification
])

classification_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train classification model
classification_model.fit(train_ds, validation_data=test_ds, epochs=10)

Epoch 1/10
    301/Unknown 1479s 5s/step - accuracy: 0.8366 - loss: 5.6392

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


301/301 ━━━━━━━━━━━━━━━━━━━━ 1561s 5s/step - accuracy: 0.8368 - loss: 5.6257 - val_accuracy: 0.8873 - val_loss: 0.2605
Epoch 2/10
301/301 ━━━━━━━━━━━━━━━━━━━━ 1568s 5s/step - accuracy: 0.9151 - loss: 0.2256 - val_accuracy: 0.9103 - val_loss: 0.2339
Epoch 3/10
301/301 ━━━━━━━━━━━━━━━━━━━━ 1534s 5s/step - accuracy: 0.9479 - loss: 0.1485 - val_accuracy: 0.9115 - val_loss: 0.3022
Epoch 4/10
301/301 ━━━━━━━━━━━━━━━━━━━━ 1556s 5s/step - accuracy: 0.9617 - loss: 0.1245 - val_accuracy: 0.9381 - val_loss: 0.2116
Epoch 5/10


KeyboardInterrupt: 

In [10]:
# Evaluate the model on the test dataset
loss, accuracy = classification_model.evaluate(test_ds)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Compute additional metrics
y_true = []
y_pred = []

for images, labels in test_ds:
    predictions = classification_model.predict(images, verbose=0)
    y_true.extend(labels.numpy())
    y_pred.extend((predictions > 0.5).astype(int).flatten())

# Generate classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=["Non-Crack", "Crack"]))

# Generate confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

53/53 ━━━━━━━━━━━━━━━━━━━━ 75s 1s/step - accuracy: 0.9258 - loss: 0.2851


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


Test Loss: 0.2445
Test Accuracy: 0.9304
Classification Report:
              precision    recall  f1-score   support

   Non-Crack       0.97      0.46      0.62       212
       Crack       0.93      1.00      0.96      1483

    accuracy                           0.93      1695
   macro avg       0.95      0.73      0.79      1695
weighted avg       0.93      0.93      0.92      1695

Confusion Matrix:
[[  97  115]
 [   3 1480]]
